In [22]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# Load ams model

In [23]:
ssp = ams.load('ams_case123_REGCV1_v7.xlsx')

ssp.LDOPF.setup()

ssp.LDOPF.om.mdl.solve()

17.88458765976425

In [24]:
ssp.LDOPF.obj.v

17.88458765976425

In [25]:
ssp.LDOPF.om.vars

OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [26]:
pg_ams = ssp.LDOPF.om.pg.value

pg_ams

array([0.17501217, 0.31501217, 0.26937836, 0.07001216, 0.08401216,
       0.04901216, 0.04201216, 0.06301216, 0.05251216, 0.05601216,
       0.05951216])

In [27]:
ssp.LDOPF.pg.get_idx()

['PV_1',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9',
 'PV_10',
 'PV_11']

# Define new variables, constraints, and obj

In [28]:
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [29]:
constraints = constriants_ams

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [30]:
# Cost data

c2 = np.array([0.01, 0.015, 0.02, 0, 0, 0, 0, 0, 0, 0, 0]) # gen quadratic cost
c1 = np.array([  20,   20,   30,  0, 0, 0, 0, 0, 0, 0, 0]) # gen linear cost

pg = ssp.LDOPF.om.pg

# pg_power = cp.power(pg, 2)
pg_power = pg**2

cost = cp.sum(c2 @ pg_power * 100 + c1 @ pg * 10 + 450)

obj = cp.Minimize(cost)

In [31]:
ssp.LDOPF.c2.v

array([0.01, 0.01, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ])

In [32]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

In [33]:
# built-in solver: ECOS_BB
# prob.solve(solver=cp.ECOS_BB, verbose=True)
# prob.solve(solver=cp.ECOS_BB)
prob.solve(solver=cp.GUROBI)

629.1747230000311

In [34]:
ssp.LDOPF.c1

RParam: c1 <GCost>, v=[20. 20. 30.  0.  0.  0.  0.  0.  0.  0.  0.]

In [35]:
ssp.LDOPF.c2

RParam: c2 <GCost>, v=[0.01 0.01 0.02 0.   0.   0.   0.   0.   0.   0.   0.  ]

In [36]:
ssp.LDOPF.om.pg.value

array([0.175 , 0.315 , 0.2695, 0.07  , 0.084 , 0.049 , 0.042 , 0.063 ,
       0.0525, 0.056 , 0.0595])

In [37]:
sum(ssp.LDOPF.om.pg.value)

1.2355000000000003

In [38]:
# check lower limit of pg
ssp.LDOPF.pmin.v

array([0.025, 0.025, 0.025, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   ])

In [39]:
# check upper limit of pg
ssp.LDOPF.pmax.v

array([0.175 , 0.315 , 2.    , 0.07  , 0.084 , 0.049 , 0.042 , 0.063 ,
       0.0525, 0.056 , 0.0595])

In [40]:
ssp.LDOPF.om.qg.value

array([ 2.   , -2.   ,  1.901, -0.2  , -0.2  , -0.2  , -0.2  , -0.2  ,
       -0.2  , -0.2  , -0.2  ])

In [41]:
ssp.LDOPF.om.ql.value

array([    0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
      

In [42]:
ssp.LDOPF.om.pl.value

array([  -2.47074166,  129.986     ,  129.93      ,  129.972     ,
        129.958     ,  129.972     ,   -2.35882731,   -2.34581751,
         16.773002  ,   20.46269779,   86.055938  ,   -0.04478477,
         -2.22714945,   21.7856158 ,   24.35686302,  -11.45555002,
        -11.44051502,    3.98366693,    4.33791189,  129.972     ,
         -2.89075766,    8.81809866,   15.33708374,   -6.83810542,
         -5.50095872,   11.73518809,   16.48576411,    7.1321384 ,
          9.72211953,   -7.93094093,   16.47639636,  -11.19736404,
        -11.21090124, -130.028     ,   -5.71484552,   32.39264999,
        -13.0993817 ,   -2.91871318,  -18.78185361,    4.12715051,
          8.06407187,  -20.97814566,   -8.49754753,    7.15543581,
          2.49920918,  -10.20762724,   33.57764479,  -15.56996403,
          5.16228644,   55.62717128,    1.82013876,   -1.41497469,
         -1.38237695,   -1.3570523 ,   18.39135643,  -12.4374427 ,
         -1.32624166,   18.39419059,  -12.44109112,   -1.29553

In [43]:
ssp.LDOPF.om.vsq.value

array([1.00249837, 1.00249852, 1.00249867, 1.00249867, 1.00249883,
       1.00249883, 1.00249898, 1.00249867, 1.00249883, 1.00249898,
       1.00249914, 1.00249929, 1.00249898, 1.00249898, 1.00249914,
       1.00249929, 1.00249944, 1.00249944, 1.00249914, 1.00249929,
       1.00249944, 1.00249929, 1.00249944, 1.00249944, 1.0024996 ,
       1.0024996 , 1.00249975, 1.00249991, 1.00249975, 1.00249991,
       1.00250006, 1.00249991, 1.00250006, 1.00250006, 1.00249929,
       1.00249929, 1.00249944, 1.0024996 , 1.0024996 , 1.00249975,
       1.00249944, 1.0024996 , 1.0024996 , 1.00249975, 1.00249975,
       1.00249991, 1.00250006, 1.00249991, 1.00250006, 1.00250006,
       1.00250021, 1.00250037, 1.00249914, 1.00249929, 1.00249944,
       1.0024996 , 1.00249975, 1.0024996 , 1.00249975, 1.00249991,
       1.00249975, 1.00249991, 1.00249991, 1.00250006, 1.00250021,
       1.00250037, 1.00250052, 1.00249991, 1.00250006, 1.00250021,
       1.00250037, 1.00250052, 1.00250006, 1.00250021, 1.00250

# 